In [190]:
# # webscraping_yahoo finance data

# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from bs4 import BeautifulSoup
# import pandas as pd
# import time

# chrome_options = Options()
# chrome_options.add_argument("--headless")

# driver = webdriver.Chrome(options=chrome_options)
# url = "https://finance.yahoo.com/quote/AAPL/history/?period1=345479400&period2=1761924776"

# driver.get(url)
# time.sleep(5)  # Wait for the dynamic table to load

# soup = BeautifulSoup(driver.page_source, "html.parser")

# # Correct table class selector based on your screenshot
# table = soup.find('table', {'class': 'table yf-1jecxey noDl hideOnPrint'})
# if table:
#     # Parse headers
#     headers = []
#     thead = table.find('thead')
#     if thead:
#         for th in thead.find_all('th'):
#             headers.append(th.text.strip())
#     # If headers list is empty, try checking the first tr's td
#     if not headers and thead:
#         first_tr = thead.find('tr')
#         if first_tr:
#             for td in first_tr.find_all('td'):
#                 headers.append(td.text.strip())

#     # Parse rows
#     rows = []
#     tbody = table.find('tbody')
#     for tr in tbody.find_all('tr'):
#         row = []
#         for td in tr.find_all('td'):
#             row.append(td.text.strip())
#         if row:
#             rows.append(row)

#     # Save to CSV
#     df = pd.DataFrame(rows, columns=headers if headers else None)
#     df.to_csv('yahoo_finance_AAPL_historical.csv', index=False)
#     print("CSV file saved as yahoo_finance_AAPL_historical.csv.")
# else:
#     print("Table not found.")

# driver.quit()

In [191]:
# # cleaning file

# import pandas as pd
# from datetime import datetime

# # Read the CSV file
# file_path = 'yahoo_finance_AAPL_historical.csv'
# df = pd.read_csv(file_path)

# # Remove rows that have 'Dividend' in any column
# df = df[~df.apply(lambda row: row.astype(str).str.contains('Dividend').any(), axis=1)]

# # Process each row
# def clean_row(row):
#     # Fix date format: "Oct 31, 2025" -> "Oct-31-2025"
#     try:
#         row[0] = datetime.strptime(row[0], "%b %d, %Y").strftime("%b-%d-%Y")
#     except Exception:
#         pass  # Skip rows where date is not in expected format
    
#     # Clean volume: "37,689,790" -> 37689790 (on last col)
#     try:
#         volume_str = str(row[-1]).replace(",", "")
#         if volume_str.isdigit():
#             row[-1] = int(volume_str)
#     except Exception:
#         pass  # If conversion fails, leave as is
#     return row

# df = df.apply(clean_row, axis=1)

# # Save the cleaned DataFrame back to the CSV file
# df.to_csv(file_path, index=False)
# print("Cleaned data saved to", file_path)


In [192]:
# data reading
import pandas as pd
from datetime import datetime

df = pd.read_csv('yahoo_finance_AAPL_historical.csv')

In [193]:
# checking data types
for col in df.columns:
    print(col, type(df[col].iloc[0]))

# null value handling
df.interpolate(method='linear', inplace=True)
null_counts = df.isnull().sum()
print(null_counts)

Date <class 'str'>
Open <class 'numpy.float64'>
High <class 'numpy.float64'>
Low <class 'numpy.float64'>
Close <class 'numpy.float64'>
Adj_Close <class 'numpy.float64'>
Volume <class 'numpy.float64'>
Label <class 'str'>
Date         0
Open         0
High         0
Low          0
Close        0
Adj_Close    0
Volume       0
Label        0
dtype: int64


/tmp/ipykernel_113245/4144968491.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='linear', inplace=True)


In [194]:
import pandas as pd

# Assuming df already exists and has 'Date', 'Open', 'High', 'Low', 'Close' columns

start_date = "1981-06-27"
end_date = "2000-12-31"

df_filtered = df.copy()

# Ensure correct data types
df_filtered["Date"] = pd.to_datetime(df_filtered["Date"], errors="coerce")
numeric_cols = ["Open", "High", "Low", "Close"]
df_filtered[numeric_cols] = df_filtered[numeric_cols].apply(pd.to_numeric, errors="coerce")

# Drop rows with missing or invalid data
df_filtered = df_filtered.dropna(subset=["Date"] + numeric_cols)

# Filter by date range
df_filtered = df_filtered[(df_filtered["Date"] >= start_date) & (df_filtered["Date"] <= end_date)]

# Prepare and save data
df_to_save = df_filtered[["Date", "Open", "High", "Low", "Close","Label"]].copy()
df_to_save["Date"] = df_to_save["Date"].dt.strftime("%Y-%m-%d")
df_to_save.rename(columns={"Date": "time"}, inplace=True)

df_to_save.to_csv("chart.csv", index=False)

print(f"✅ Saved chart.csv successfully with columns: time, Open, High, Low, Close for date range {start_date} to {end_date}")

✅ Saved chart.csv successfully with columns: time, Open, High, Low, Close for date range 1981-06-27 to 2000-12-31
